# [ER-190C] Homework 10: Classification Trees
----


### Table of Contents

[Introduction](#intro) <br>
[1. The Data](#data) <br>
[2. Decision Trees From Scratch](#scratch) <br>
[3. Implementing with Scikit-learn](#sk) <br>
[4. Ensemble Methods](#improve) <br>
[5. Comparing Methods](#compare) <br>

**Dependencies:**

In [ ]:
import urllib
import os.path
from shutil import copyfile

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")

In [ ]:
!pip install xlrd
!pip install graphviz

----

## Introduction <a name = 'intro'></a>

Decision trees are a powerful prediction method and are easy to interpret. A decision tree can explain exactly why a specific prediction was made. They usually aren't used by themselves, but many trees are used for ensemble methods such as random forests and bagging. Trees are, overall, a relatively more accessible method for predictive modeling since they are used for both regression and classification and can take in both continuous and categorical data.

In this homework, we'll be doing a brief exploration of the CalEnviroScreen dataset, testing out how to make a tree from scratch, as well as implementing various ensemble methods using scikit-learn. It'll be a comprehensive survey of trees and the multitude of algorithms that arise from one tree!


----

## 1. The Data <a name='data'></a>

In this homework, we will be working with the [California Communities Environmental Health Screening Tool (CalEnviroScreen)](https://oehha.ca.gov/calenviroscreen/report/calenviroscreen-30), which uses demographic and environmental information to identify communities that are susceptible to various types of pollution. The various variables in this dataset contribute to the CES score, which reflects a community's environmental conditions and its vulnerability to environmental pollutants.

We worked with this dataset in lab and this will be one of two homeworks that utilize this dataset.  Although this dataset scores around 8000 of census tracts in California, we'll focus on a subset which makes the decision tree slightly more interpretable. 

In [ ]:
def getfile(filename):
    
    if os.path.isfile(filename) == False: # check if you've got the file, if not, download it
        weborlocal = input('File is not in the present working directory.  Get it from the web, or local? ')
        
        if weborlocal == 'web':
            url = input('What is the url? ')
            urllib.request.urlretrieve(url, filename);
        
        elif weborlocal == 'local':
            directory = input('What\'s the file\'s directory? ')
            print(directory+filename)
            
            if os.path.isfile(directory+filename)==True:
                copyfile(directory+filename, filename)
            else:
                print('Can\'t find the file.  Check the directory and make sure the path ends in "/".')
        else:
            print('Please choose "web" or "local".  Try again.')
    
    else:
        print('That file is in the present working directory.')

In [ ]:
filename = 'ces3results.xlsx'
getfile(filename)

We've just obtained an excel spreadsheet of environmental data from the California EnviroScreen data set. Now, we will format the spreadsheet into a DataFrame object in order to explore its properties. 

Documentation on Pandas' excel methods can be found at [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html#pandas.read_excel).

First we will focus on the first sheet, which contains the data that we'll work with in this homework; run the following cell to look at the four sheets in this excel file.

In [ ]:
xl = pd.ExcelFile(filename)
print(xl.sheet_names) # display a list of the sheets in the spreadsheet

<div class="alert alert-warning">

<b>Question 1.1:</b> Load the first sheet of the Excel file and assign it to the variable `df0`. 

</div>

In [ ]:
...
df0.head()

Before we explore this dataset, look at some of the features -- notice that this dataset doesn't include the units of most of it's features. Let's take a look at a different sheet.

Run the following cell to load the data dictionary.

In [ ]:
dd = xl.parse('Data Dictionary').rename(columns = {'CalEnviroScreen 3.0: Data Dictionary':'Variable', 
                                                   'Unnamed: 1': 'Description'})
dd.head(10)

Before we move on, let's check the sheet we just loaded. 

<div class="alert alert-warning">

<b>Question 1.2:</b> Does the number of columns in the first sheet correspond with the number of rows of the variables in the table we just created? Your answer should return a boolean value.

</div>

In [ ]:
...

Now that we've checked if the features and columns correspond, let's move on and take a closer look at the description of these variables.

<div class="alert alert-warning">

<b>Question 1.3:</b> In the `dd` dataframe, drop the rows that contain `Pctl` in the variable name and drop the last column  as well. Make sure the shape is correct.

Note: If you don't pass, rerun the cell that loads this dataframe (two cells before this one)

</div>

In [ ]:
...
...

assert dd.shape == (35, 2 )
dd.head()

It seems that not all of the variables have an explicit description of the unit of measurement, or are a scaled or weighted metric. Regardless, we have an idea of the variables we are working with and their units of measurement.

Let's move on and take a look at the data for the city of Berkeley.

<div class="alert alert-warning">

<b>Question 1.4:</b> Find all instances that Berkeley appears in the dataset and assign it to the variable `berkeley`. Then, select the columns `Census Tract`, `ZIP`, `CES 3.0 Score`, `Housing Burden`, `Poverty`, `Pesticides`, `Groundwater Threats`, and `Pollution Burden`, and assign the resulting table to `berkeley_ft`.

</div>

In [ ]:
berkeley = ...
berkeley_ft = ...
berkeley_ft.head(10)

Are these values expected or slightly surprising? If we were working with percentiles, it would give us a better idea of how impacted (or not) a specific community is compared to the rest of the tracts in California. We can plot these values in a histogram later to evaluate how "high" or "low" a value is compared to the rest.

Let's compare the two rows in our table -- specifically the rows with Census Tract `6001421100` and `06001422000`. Run the following cell to load the two rows.

In [ ]:
ind1 = berkeley_ft[berkeley_ft['Census Tract'] == 6001421100.0]
ind2 = berkeley_ft[berkeley_ft['Census Tract'] == 6001422000.0]
ind1.append(ind2)

These two communities are extremely different -- one has an extremely low score and one has a high score, and the values in the features are also pretty varied. Let's take a look at where their scores lie in various histograms of the features.

<div class="alert alert-warning">

<b>Question 1.5:</b> Plot a distribution for the `Poverty` variable of all Berkeley tracts as well as for `Pollution Burden`. Along with these histograms, plot the points in which these two tracts lie within the distribution.

Hint: Use `plt.scatter` for the individual points.

</div>

In [ ]:
...
...
...

In [ ]:
...
...
...

Lastly, let's take a look at the demographic profiles of these two communities. This data is stored in the `Demographic profile` sheet.

<div class = "alert alert-warning">

<b>Question 1.6:</b> Similar to what we did earlier by comparing the two rows, compare the two tracts' demographic information (i.e. display the two rows together). We've loaded the sheet into a dataframe `dp` and have renamed the columns for you.

</div>

In [ ]:
dp = xl.parse('Demographic profile')
dp = dp.rename(columns = {'Census Tract ': 'Census Tract',
                          'Age group from 2010 Census (%)':'Children < 10 (%)', 
                          'Unnamed: 7':'Pop 11-64 years (%)',
                          'Unnamed: 8':'Elderly > 65 (%)', 
                          'Race or ethnicity from 2010 Census (%)':'Hispanic (%)',
                          'Unnamed: 10':'White (%)',
                          'Unnamed: 11':'African American (%)', 
                          'Unnamed: 12': 'Native American (%)', 
                          'Unnamed: 13':'Asian American (%)', 
                          'Unnamed: 14': 'Other (%)'})

# YOUR CODE HERE
demo1 = ...
demo2 = ...
demo1.append(demo2)

If you're curious, you can take a look at where these census tracts are located by inputting the coordinates in [maps](https://www.google.com/maps).
- Coordinates for the first row: (+37.8994340, -122.2661928)
- Coordinates for second row: (+37.8590327, -122.3013426)

You can then find the tract areas by using this [web tool](https://data.cityofberkeley.info/Demographics/Census-Tract-Polygons-2010/peq3-2arw) provided by the city of Berkeley!

----

## 2. A Decision Tree From Scratch <a name = 'scratch'></a>




Let's start by creating a decision tree from scratch! Even though trees are pretty easy to interpret, there's a lot that runs to create the final tree. We'll walk through creating a tree which will hopefully help our intuition when we use scikit-learn later in this homework.

First, let's take a very small subset of our data so that our tree will be easy to work with. There are a few things we're doing here to make this process more digestible -- we're taking a sample of 10 from the table and we are classifying the top and bottom groups of the CES dataset.

<div class="alert alert-warning">

<b>Question 2.1:</b> In the following cell, we've loaded the first sheet of the excel file into a dataframe called `data`. Find the indices of the group with the highest scores and the lowest scores as well, assigning them to the variables `top` and `bottom` respectively. Then, fill out the ellipses in the `sample` method  -- we want to take a sample of 10, and set the random_state to 1.

</div>

In [ ]:
data = data = df0.rename(columns={'CES 3.0 \nPercentile Range':'CES 3.0 Percentile Range'})

...
...

scratch = data.iloc[top.append(bottom)].dropna()
scratch = scratch[["Groundwater Threats", 
                   "Drinking Water",           # YOUR CODE HERE
                   "CES 3.0 Percentile Range"]].sample(...)
scratch

The first the we want to do when we building a tree is how can we create each split in the tree. To do so, we'll start with calculating the gini index. The optimal split for any given feature is determined by a cost function that defines purity. The gini index quantifies a split by the class composition of the resulting two groups where a low score means a near perfect split and a high score means the composition is nearly the same in both groups. The gini index is calculated as follows:

$$ G = \sum_{k = 1}^{K}{\hat{p}_k(1 - \hat{p}_k)} = \sum_{k = 1}^{K} (1 - \hat{p}^2_k)$$

where $G$ is the Gini index, and $\hat{p}_k$ is the proportion of count in each variable.

<div class="alert alert-warning">

<b>Question 2.2:</b> Take a look at the following cell, which contains the code in order to calculate the gini index. There are letters following three pound symbols. Write down what the line below is doing for each letter.

</div>

In [ ]:
def gini_index(groups, classes):
    ### A ###
    n_instances = float(sum([len(group) for group in groups]))
    ### B ###
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        ### C ###
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        ### D ###
        gini += (1.0 - score) * (size / n_instances)
    return gini

***YOUR ANSWERS HERE***

**A)**

**B)**

**C)**

**D)**

------

Now, we can look at creating a split. A split is comprised of an attribute in the dataset and a value and creating one involves three parts.

1. Calculating the gini index
2. Splitting a dataset
3. Evaluating all splits

We'll look at the second step now!

Splitting a dataset means separating a dataset into two lists of rows given the index of an attribute and a split value for that attribute. Once we have the two groups, we can use the gini index above to evaluate the cost of the split. In this procedure, we iterate over each row to check if the attribute value is below or above the split value and then we'll assign it to the left or right group respectively.

Run the following cell to load the function.

In [ ]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

<div class="alert alert-warning">

<b>Question 2.3:</b> What does the group sorted in the right list represent?

</div>

***YOUR ANSWER HERE***

----
We can create our splits now. Given a dataset, we must check every value on each attribute as a candidate split, evaluate the cost of the split and find the best possible split we could make. The lowest gini score across all the features would then be chosen as the best split and a node would be created.

We will use a dictionary to represent a node in the decision tree as we can store data by name. When selecting the best split and using it as a new node for the tree we will store the index of the chosen attribute, the value of that attribute by which to split and the two groups of data split by the chosen split point.

The function below runs this procedure. Run the cell to load function.

In [ ]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index = index
                b_value = row[index]
                b_score = gini
                b_groups = groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

We have all of the tools to find the best splits of the tree -- let's see how we can use them to *build* one from the ground up (or from the top since trees are inverted...). 

Building a tree takes two main steps: finding leaves (e.g. when to stop the tree) and recursively splitting the tree.

The following cell contains the function that returns the most common output in a list of rows.

In [ ]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

<div class="alert alert-warning">

<b>Question 2.4:</b> What is a problem when fitting a tree too deeply? What about having too many nodes?

</div>

***YOUR ANSWER HERE***

----

We're almost there! The following cell contains the function that performs the recursive splitting. 

Here are the steps of this procedure:

1. Two groups of data split by the node are extracted and deleted from the node. The node no longer requires access to these data when we work on the node.
2. We check if either left or right group of rows is empty and if so we create a terminal node using the data and computed scores we have.
3. Check if we have reached our maximum depth. If we have, create a terminal node.
4. If the group of rows is too small, we process the left child and create a terminal node. Otherwise, we create and add the left node in a depth-first fashion until the bottom of the tree is reached on this branch.
5. Right side is then processed in the same manner, as we rise back up the constructed tree to the root.

<div class="alert alert-warning">

<b>Question 2.5:</b> Similar to question 2.2, look at the following cell and note what each control case is doing in the `split` function.

</div>

In [ ]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    ### A ###
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    ### B ###
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    ### C ###
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    ### D ###
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

***YOUR ANSWER HERE***

**A)**

**B)**

**C)**

**D)**

----

Awesome! We're ready to build our tree. In the following cell, we have our `build_tree` and `print_tree` function. Most of the steps were abstracted away in `get_split` and `split` functions. Before we use the function, we can't directly use a pandas dataframe or series as input to this function.

<div class="alert alert-warning">

<b>Question 2.6:</b> In the same cell, assign a list of the rows of the dataframe to the variable `rows`. Run the cell and see the resulting tree. What do the inqualities represent? How does it relate to the data we inputted into the tree?

</div>

In [ ]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

# YOUR CODE HERE       
rows = ...
scratch_tree = build_tree(rows, 2, 2)
print_tree(scratch_tree)

***YOUR ANSWER HERE***

----

Just for fun, we can also use this tree for prediction (although we really shouldn't since we used the full ten samples to fit the tree). Run the following cell to see the expected and predicted values from our dataset!

In [ ]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

for row in rows:
    prediction = predict(scratch_tree, row)
    print('Expected: ', row[-1])
    print('Got: ', prediction, '\n')

Now that we have a better idea of how a tree works under the hood, let's move on to scikit-learn and create a more complex tree.

---

## 3. Implementing with Scikit-Learn <a name = 'sk'></a>

Since our data has quite the amount of features, creating and tuning a tree from scratch would be a hassle and would not necessarily lead to optimal results. Here's where scikit-learn comes in and streamlines the process for us.

We'll be using scikit-learn's `DecisionTreeClassifier` ([documenation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)) in this section and use a larger subset of the data to create a more complex model.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import tree

data.head()

----

### 3.1. The First Tree

For our first model, we'll use the features to predict which score bracket a certain tract lies in, namely we want to use these features to predict the `CES 3.0 Percentile Range`. For the sake of interpretability, we'll focus on the 91th percentile and above, and up to the 10th percentile of scores.

<div class="alert alert-warning">

<b>Question 3.1.1:</b> Before we move on, what is a problem that can stem from using such a specific subset of our data to build a classifier?

</div>

***Your Answer Here***

Let's take a look at the scores that have been binned already in the dataset so we know how to set up our model. Run the following cell to see them!

In [ ]:
data['CES 3.0 Percentile Range'].unique()

It looks like we have a total of four values that our tree can potentially classify if we want to take the top and bottom 10% of scores.

<div class="alert alert-warning">

<b>Question 3.1.2:</b> Get the index values for the data in the `96-100% (highest scores)` and `91-95%` range, and assign the values to the variable `pct90`. Do the same for `6-10%` and `1-5% (lowest scores)`, assigning the resulting values to the variable `pct10`.

<br>

Then, use `pct90` and `pct10` to get the resulting table and reassign it to `data`. This table should only contain the four ranges in the column `CES 3.0 Percentile Range`

</div>

In [ ]:
...
print(pct90)

...
...
...
print(pct10)

data = ...

assert len(data['CES 3.0 Percentile Range'].unique()) == 4

data.head()

Now that we have our desired observations, let's filter through the certain features we want to use in our classification tree. Remember that a tree can use both continuous and categorical data, but we probably don't want to work with features like the county or zip code since it would clutter our data if we wanted to encode them. Run the following cell to drop the columns as well as the NaN values.

In [ ]:
data = data.drop(columns = ['Census Tract', 'CES 3.0 Score', ' CES 3.0 Percentile', 
                            'California County', 'ZIP', 'City', 'Longitude', 'Latitude'])
data = data.dropna()
data.head()

The last thing we need to do before we create our decision tree is to drop the `Pctl` columns, as well as create the testing, training, and validation sets.

<div class="alert alert-warning">

<b>Question 3.1.3:</b> Drop all columns that contain the string `Pctl` for our set of features. Don't forget to drop the percentile range column. We don't want that in our features! 

Assign the resulting table the variable `features` and assign the percentile ranges to the variable `target`.

Lastly, create the training, testing, and validation sets by splitting the set into training and testing first, then split the training test. Use `random_state = 1` both times, and create an 80/20 train/test split, and a 75/25 train/validation split.

</div>

In [ ]:
drop_columns = ...
...

features = ...
target = ...

# split test set
X, X_test, y, y_test = train_test_split(...)

# split between train and validation sets
X_train, X_val, y_train, y_val = train_test_split(...)

Awesome! We can finally create a tree! 

<div class="alert alert-warning">

<b>Question 3.1.3:</b> Instantiate a `DecisionTreeClassifer` model and call it `first_tree`. Fit the model using the training data, and score it using both the training and validation set. Assign the scores to the variables `train_score` and `val_score` respectively.

</div>

In [ ]:
...
...

print("Number of features: {}".format(first_tree.tree_.n_features))
print("Number of nodes (leaves): {}".format(first_tree.tree_.node_count), "\n")

...
...

print('Train Score: ', train_score)
print('Validation Score: ', val_score)

The score isn't too shabby for a tree that was put together pretty quickly. The nice part about decision trees is that they're easy to visualize and interpret and with scikit-learn, we can export the an image of the tree. Unfortunately, due limitations on DataHub, we can't output an image directly in a notebook. 

Luckily, we can copy the code and visualize the tree on [Webgraphviz](http://webgraphviz.com). By running the following cell, you'll see a pretty long output -- follow the link and copy and paste the output to get a visualization of the decision tree we fit!

In [ ]:
import graphviz
print(tree.export_graphviz(first_tree, feature_names=X.columns))

<div class="alert alert-warning">

<b>Question 3.1.4:</b> What does each line in the box mean?

</div>

***YOUR ANSWER HERE*** <br>
First line:

Second line:

Third line:

Fourth Line:

<div class="alert alert-warning">

<b>Question 3.1.4:</b>   Based on the visualization, can you see any problems or pitfalls with the model we just created? Can you infer the "most and least important" features?

</div>

***YOUR ANSWER HERE***

----
<a name='first'></a>
With scikit-learn, we're also able to check the feature importances. Running the following cell, we can see the features and the their importance based on the data we used to fit the tree -- this can be helpful when tuning or pruning the tree!

In [ ]:
pd.DataFrame({'Feature': X.columns, 'Importance': first_tree.feature_importances_})

-----

### 3.2. Tuning hyper parameters and Pruning

We've had a good look at what we can do with a decision tree in using scikit-learn. Let's dive into learning about the hyperparameters, which can help improve our classification model. Again, the documentation for `DecisionTreeClassifier` is linked [here](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

If you noticed in the tree earlier, there were a *lot* of nodes. Some of these nodes have a gini score of 0 or have small sample numbers. We can overcome these problems by tweaking the hyperparameters of the tree. 

In the following cell, we have almost the exact same code from earlier, but we've added two parameters: `max_leaf_nodes` and `max_features`. Let's try two or three sets of different values for these two parameters and see how much we can improve our original model.

In [ ]:
tuned_tree = DecisionTreeClassifier(max_leaf_nodes=..., max_features=...)
tuned_tree.fit(X_train, y_train)

print("Number of features: {}".format(tuned_tree.tree_.n_features))
print("Number of nodes (leaves): {}".format(tuned_tree.tree_.node_count),"\n")

tuned_train_score = tuned_tree.score(X_train, y_train)
tuned_val_score = tuned_tree.score(X_val, y_val)

print('Train Score: ', tuned_train_score)
print('Validation Score: ', tuned_val_score)

Were you able to obtain validation score higher than what we had originally? It probably took some time to figure out which values were better than others.

Fortunately, we don't have to test various parameters by manually inputting them and running the cell, which would be exhausting! Instead, we can use [`RandomizedSearchCV` from scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) to help us find optimal parameters. 

It takes in a model, a distribution of parameters we want to test in the model, and other parameters that we can tweak. We'll only use `cv` and `n_iter` along with the two required arguments. 

The parameter distribution is a dictionary that takes in the parameter name as a key and a range of random values that we want to test. We'll be using `randint` from scipy.stats. Run the following cell -- we will also fit the CV search which will take some time.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {'max_leaf_nodes': randint(3, 100),
              'max_features': randint(2, 25),
              'max_depth': randint(1, 10)}

rnd_search = RandomizedSearchCV(tuned_tree, param_distributions=param_dist, 
                                cv=10, n_iter=200)
rnd_search.fit(X_train, y_train)

After fitting our data, we can check the score and the values for the parameters that return the "best" score. Run the following cell to get these values!

In [ ]:
print(rnd_search.best_score_)
print(rnd_search.best_params_)

<div class="alert alert-warning">

<b>Question 3.2.1:</b> With our new parameters, set the parameters of `tuned_tree` to the ones that we found using the randomized search. Then, score the model using the training and validation sets.

</div>

In [ ]:
...
...

tuned_train_score = ...
tuned_val_score = ...

print('Train Score: ', tuned_train_score)
print('Validation Score: ', tuned_val_score)

Awesome! Hopefully this tuned tree performed better than our original one (due to the random samples, it's hard to say). Let's also print out the feature scores like we did initially. Do you see a difference between the feature importances of the tuned tree compared to those of the first tree?

In [ ]:
pd.DataFrame({'Feature':X_train.columns, 'Importance': tuned_tree.feature_importances_})

Lastly, for this tuned tree, let's take a look at how adjusting the hyperparameters altered the decision tree. Like last time, copy and paste the code into [Webgraphviz](http://webgraphviz.com) to visualize the tree.

In [ ]:
print(tree.export_graphviz(tuned_tree, feature_names=X.columns))

----

We briefly covered hyperparameter tuning -- let's move on to pruning our tree. Unfortunately, pruning is not a feature built into scikit-learn yet, but we can use the structure of the tree (which is readily available by calling `first_tree.tree_`) to traverse its nodes.

The following is a function that prunes leaves and nodes based on how many samples we want a node to have.

In [ ]:
def prune(decision_tree, min_samples_leaf = 1):
    decision_tree.min_samples_leaf = min_samples_leaf
    tree = decision_tree.tree_
    
    for i in range(tree.node_count):
        n_samples = tree.n_node_samples[i]
        if n_samples <= min_samples_leaf:
            #sklearn.tree structure: -1 denotes a leaf
            tree.children_left[i] = -1
            tree.children_right[i] = -1

For this pruned tree, let's compare it to the `first_tree` model -- so we won't set any specific hyperparameters.

<div class="alert alert-warning">

<b>Question 3.2.2:</b> Instantiate another model and assign it to `pruned_tree`. Fit it with the training data and then prune the tree -- try your best to find an optimal value. It doesn't necessarily have to be high, but try to find a better score than our initial model.

</div>

In [ ]:
...
...

print(sum(pruned_tree.tree_.children_left < 0))

...

print(sum(pruned_tree.tree_.children_left < 0), '\n')

print('Train Score: ', pruned_tree.score(X_train, y_train))
print('Validation Score: ', pruned_tree.score(X_val, y_val))

Let's also take a look at the feature importances. Do they look any different from the previous two models?

In [ ]:
pd.DataFrame({'Feature':X.columns, 'Importance': pruned_tree.feature_importances_})

Run the following cell and copy and paste the output on [webgraphviz](http://webgraphviz.com).

In [ ]:
print(tree.export_graphviz(pruned_tree, feature_names = X.columns))

<div class="alert alert-warning">

<b>Question 3.2.3:</b> Take a look at each of the three trees. What did adjusting the hyperparameters do to the tree? What about the pruning? Does it make your model easier to understand?

</div>

***YOUR ANSWER HERE***

----

### 3.3. Binning

Let's create another decision tree and use specific features to predict the environmental conditions of these communities. 

Take a look at the columns of the table. Notice that a lot of the features that we used in our first classification tree are *continuous*, not *discrete*. Let's say we wanted to make some sort of predictive model, using only those continuous values. Instead of predicting how impacted these communities are based on whichever group they fall under, we would be predicting their scores, which might not necessarily be helpful if we wanted to categorize these groups.

What we can do to work around this is put certain scores into bins, or ranges, of values. In order to do so, we can either use the score of the characteristic or the percentile. Like we did in the first tree, we'll use the percentile, which gives us a better idea of how impacted a certain community is compared to other communities in this dataset.

Run the following cell to take a subset of our data. For this section, our target variable will be `Pesticides Pctl`.

In [ ]:
columns = ['Pesticides Pctl', 'Linguistic Isolation', 'Education', 'Unemployment', 
           'Poverty', 'Housing Burden']
pesticides = data[columns]
pesticides.head(10)

We can put the values in `Pesticides Pctl` into bins to "classify" a community's exposure to pesticides given certain demographic characteristics.

<div class="alert alert-warning">

<b>Question 3.3.1:</b> Take the top and bottom 20th percent of our data. Reassign the resulting dataframe to `pesticides`.

</div>

In [ ]:
...

Since the values in `Pesticides Pctl` is still continuous, let's put them into two bins: the bottom 20% will go into the bin `0-19.99%` and the top 20% will go into `80-100%`.

<div class="alert alert-warning">

<b>Question 3.3.2:</b> Complete the for loop to put each observation into their respective category. 

Make sure that your code is correct by verifying that the percentile group for the first 10 rows correspond with the percentile in the first column.

</div>

In [ ]:
bins = ['0-19.99%', '80-100%']
binned = []
for pctl in pesticides['Pesticides Pctl'] // 10:
    ...


...
pesticides.head()

Now that we have our categorized numerical values, we can fit a model and make some predictions! Run the following cell to fit and score a decision tree with the data we crafted.

In [ ]:
p_X = pesticides.drop(columns=['Pesticides Pctl', 'Pesticides Binned'])
p_y = pesticides['Pesticides Binned']

p_X_train, p_X_test, p_y_train, p_y_test = train_test_split(p_X, p_y, random_state = 2,
                                                           test_size=.25)
second_tree = DecisionTreeClassifier()
second_tree.fit(p_X_train, p_y_train)

print("Number of features: {}".format(second_tree.tree_.n_features))
print("Number of nodes (leaves): {}".format(second_tree.tree_.node_count), '\n')

print('Train Score: ', second_tree.score(p_X_train, p_y_train))
print('Validation Score: ', second_tree.score(p_X_test, p_y_test))

Let's move on to more improvements we can make to decision tree models.

----

## 4. Ensemble Methods <a name = 'improve'></a>

In the previous section, we mainly were focusing on fitting one tree and editing it's hyperparameters or pruning it. Otherwise, we do anything more to alter it. Also, you may have noticed that, although the training scores were  extremely high, the validation scores usually weren't as high. This is where ensemble methods come in, namely bagging, random forests, and boosting. We call them "ensemble methods" because we fit a many trees and, for example in bagging, we take the average of all of the models as a means to reduce the problem of overfitting.

----

### 4.1. Bagging

Bagging takes in $B$ different bootstrapped training data sets and we train our tree on each $b$th training set to get $\hat{f}^{*b}(x)$. Then, we average all of the predictions to get 

$$\hat{f}_{bag}(x) = \frac{1}{B} \sum_{b=1}^{B}\hat{f}^{*b}(x)$$

In terms of classifying, bagging records the class from each bootstrapped sample and chooses the most commonly occurring majority class among the B predictions.

Let's proceed by using `BaggingClassifier` from scikit-learn's ensemble module ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html)). 

<div class="alert alert-warning">

<b>Question 4.1.1:</b> Use the training data and fit the bagging classifier. Score the model -- how much has the score improved?

</div>

In [ ]:
from sklearn.ensemble import BaggingClassifier

...
...

bag_train_score = ...
bag_val_score = ...

print('Train Score: ', bag_train_score)
print('Validation Score: ', bag_val_score)

Like before, we can use the randomized parameter search to find optimal parameters for our model.

<div class="alert alert-warning">

<b>Question 4.1.2:</b> Create a dictionary for the parameter distribution and fit a `RandomizedSearchCV` with `cv = 10` and `n_iter = 50`.

</div>

In [ ]:
param_dist = {}

rnd_search = RandomizedSearchCV(...)

...

print(rnd_search.best_params_)

<div class="alert alert-warning">

<b>Question 4.1.2:</b> Set the parameters in `bag_tree` and re-score the model.

</div>

In [ ]:
...

bag_train_score = ...
bag_val_score = ...

print('Train Score: ', bag_train_score)
print('Validation Score: ', bag_val_score)

We see that bagging the model has a better score than the first model we created. Even if we get a better model in the end, there are still some consequences.

<div class="alert alert-warning">

<b>Question 4.1.3:</b> What are some drawbacks of bagging compared to a single decision tree? Is there a way to compare the quality of a single decision tree model compared to a bagged model?

</div>

***YOUR ANSWER HERE***

----

### 4.2. Random Forests

Random forests take a similar approach to bagging, in which we fit various decision trees on resampled data. But, when each tree is constructed, not every feature is considered as split candidates for each decision point; we only take a subset of the total predictors in the model.

When building a random forest compared to a single decision tree, adding randomization into the features that create the model, then averaging the predictions across models will typically produce a model "decorrelates" the trees and in turn is more reliable for prediction.

We'll use scikit-learn's `RandomForestClassifier()` to implement our model. The documentation can be found [here](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

<div class="alert alert-warning">

<b>Question 4.2.1: </b>  Create a default RandomForestClassifier and same as before, fit the training data and score the model.

</div>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

...
...

rf_train_score = ...
rf_val_score = ...

print('Train Score: ', rf_train_score)
print('Validation Score: ', rf_val_score)

<div class="alert alert-warning">

<b>Question 4.2.2: </b>  We'll use RandomizedSearchCV again to find values for our hyperparameters. Look at the documentation to see which parameters we can find using the random search. Then, find the cross-validated parameters and print them. 

Note: This might take some time, so feel free to move onto the next question while it's working.
</div>

In [ ]:
param_dist = ...

...

...

print(...)

We see that we can choose the number of features or predictors that the random forest will consider in each tree. 

<div class="alert alert-warning">

<b>Question 4.2.3: </b>  What if the number of predictors we set is equal to the total number predictors in our regular model? Which method would it be equivalent to?

</div>

***YOUR ANSWER HERE***

<div class="alert alert-warning">

<b>Question 4.2.4: </b> Once the search has completed and you've printed the parameters, set them in the `rf_tree` model and re-fit the model, and print out the scores.
</div>

In [ ]:
...
...

rf_train_score = ...
rf_val_score = ...

print('Train Score: ', rf_train_score)
print('Validation Score: ', rf_val_score)

Similar to bagging, we cannot visualize a random forest; with a reduction in variance comes at a price of interpretability. 

----

### 4.3. Boosting

Lastly, we'll cover boosting, which is yet another approach to improve a decision tree model. The boosting approach grows a tree slowly. Each boosting algorithm has a slightly different approach, but the general idea is the same behind each one.

First, we'll briefly cover AdaBoost (for ADAptive BOOSTing). The adaptive part of the algorithm comes from how it updates the data for each weak model in the sequence; it combines many relatively weak and inaccurate classifiers. So it shines when a regular classifier doesn't perform well with a given dataset.

We can use `AdaBoostClassifier` from scikit-learn to try this out. Run the following cell to see how it performs.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_tree = AdaBoostClassifier()
ada_tree.fit(X_train, y_train)

ada_train_score = ada_tree.score(X_train, y_train)
ada_val_score = ada_tree.score(X_val, y_val)

print('Train Score: ', ada_train_score)
print('Validation Score: ', ada_val_score)

Oops. It doesn't look too great. One thing with AdaBoost is that it doesn't handle noisy data or outliers well. In this case, our data is basically polarized -- we're taking values from the top and bottom 10% -- which means it might not work as well as we want it to.

We can turn to gradient boosting, which is another boosting method. Gradient boosting involves three elements:
1. A loss function to be optimized.
2. A weak learner to make predictions.
3. An additive model in which we add weak learners to minimize the loss function.

We'll use scikit-learn's `GradientBoostingClassifier` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)) to see how it performs with our data.

<div class="alert alert-warning">

<b>Question 4.3.1: </b> We'll do this the last time! Fit the training data to the GradientBoostingClassifier and score the model. How much did it improve over the original model?

</div>

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

...
...

gb_train_score = ...
gb_val_score = ...

print('Train Score: ', gb_train_score)
print('Validation Score: ', gb_val_score)

We can also improve the model by tuning the hyperparameters. Since we've done this quite a bit, we've coded the cell for running the search -- just run the following cell!

In [ ]:
param_dist = {'learning_rate': randint(1, 5),
              'max_leaf_nodes': randint(3, 100),
              'max_features': randint(2, 27),
              'max_depth': randint(1, 10),
              'min_samples_leaf': randint(1, 30)}

rnd_gb_search = RandomizedSearchCV(gb_tree, param_distributions=param_dist, 
                                cv=10, n_iter=50)

rnd_gb_search.fit(X_train, y_train)

print(rnd_gb_search.best_params_)

<div class="alert alert-warning">

<b>Question 4.3.2: </b> Once the search is complete, set the parameters and re-fit and score the model.

</div>

In [ ]:
...
...

gb_train_score = ...
gb_val_score = ...

print('Train Score: ', gb_train_score)
print('Validation Score: ', gb_val_score)

With boosting, we can also find the relative feature importance with this boosted model. This can help with the feature importance like before. 

<div class="alert alert-warning">

<b>Question 4.3.3: </b>  Get the feature importances and calculate the *relative* feature importance, which is each feature divided by the maximum feature multiplied by 100. Load the data into a table.
</div>

In [ ]:
...
...
...

<div class="alert alert-warning">

<b>Question 4.3.4: </b> Let's take a look at how these features compare with each other. Sort the values in the dataframe and create a bar chart that displays the feature importances in descending order.

</div>

In [ ]:
...

<div class="alert alert-warning">

<b>Question 4.3.5: </b> Lastly, compare these values with the values we computed the [very first time](#first). Are these values similar? If not, which one does a better job indicating which features are more important?

</div>

***YOUR ANSWER HERE***

----

We've gone through many methods of creating a decision tree and tuning and improving it, as well as various algorithms that use multiple trees to create a more reliable tree for prediction. Even though we've primarily have been testing our models with the training and validation set, we would still need to cross validate each of these models to see which one is the optimal one to choose given our data and how the model performs (this is even more important when a lot of the scores turned up around 92-93%). 

Before you finish up this homework, run the following cell to see which has the highest score with our test set!

In [ ]:
models = [first_tree, tuned_tree, pruned_tree, bag_tree, rf_tree, ada_tree, gb_tree]
for i in models:
    print('Test Score: ', i.score(X_test, y_test))

----

## Submission

Congrats, you're done with homework 10! 

In order to turn in this assignment, go to the toolbar and click **File** -> **Download as** -> **.html** and **.ipynb**. Submit the files through bCourses.

----

Notebook developed by: Jason Jiang

Data Science Modules: http://data.berkeley.edu/education/modules